In [2]:
%matplotlib inline
import numpy as np
from gPhoton.pipeline import execute_pipeline
import pandas as pd
import astropy.units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
from gPhoton.io.fits_utils import get_fits_data, read_wcs_from_fits
from astropy.visualization import ZScaleInterval
import os
import tqdm
import imageio.v2 as imageio
from IPython.display import Image
import astropy.wcs
from gPhoton.coadd import make_shared_wcs, project_to_shared_wcs, bin_projected_weights

Notebook summary: further work on photometry. largely duplicates _008

Pipeline steps:
1. Run photometry with extraction on NUV
2. Run photometry with extraction on FUV
3. Run photometry on FUV with NUV source positions
4. Run photometry on NUV with FUV source positions
5. Generate a merged catalog from NUV extractions
6. Generate a merged catalog from FUV extractions
7. Generate a merged catalog from all NUV / FUV detections

In [3]:
from glcat_merge_utils import *
from lightcurve_utils import *

In [7]:
def accumulate_run_data(eclipse,
                        datadir = "/Users/cm/github/gphoton2_refactor/gPhoton2/test_data/"
                       ):
    edir = f"e{str(eclipse).zfill(5)}"
    apers = [1.5, 2.3, 3.8, 6.0, 9.0, 12.8, 17.3]
    data = {'eclipse':eclipse}
    at = {}
    for p in 'nf':
        for m in 'nf':
            at[f'{p}{m}'] = {}
            for a in apers:
                at[f'{p}{m}'][a] = pd.read_csv(
                    f"{datadir}{edir}/{edir}-{p}d-b00-f0120-movie-photom-{str(a).replace('.','_')}-mo{m}.csv",
                    index_col=None)
    data['phot'] = at

    data['expt'] = {'NUV':pd.read_csv(f"{datadir}{edir}/{edir}-nd-b00-f0120-movie-exptime.csv",index_col=None),
                    'FUV':pd.read_csv(f"{datadir}{edir}/{edir}-fd-b00-f0120-movie-exptime.csv",index_col=None)}
    return data
    
data = accumulate_run_data(23456)
data.keys()

dict_keys(['eclipse', 'phot', 'expt'])

In [13]:
def derived_photometry_table(data,aper,code):
    band = {'n':'NUV','f':'FUV'}[code[0]]
    expt = data['expt'][band]['expt'].sum()
    cps = np.array(data['phot'][code][aper]['aperture_sum'].tolist())/expt
    cps_err = np.sqrt(data['phot'][code][aper]['aperture_sum'].tolist())/expt
    mag = counts2mag(cps,band)
    mag_err = np.abs(counts2mag(cps-cps_err,band) - mag)
    return pd.DataFrame(
        {'CPS':cps,'CPS_ERR':cps_err,
         'MAG':mag,'MAG_ERR':mag_err,
         'MASK_FLAG':(data['phot'][code][aper]['aperture_sum_mask'] != 0).astype(int),
         'EDGE_FLAG':(data['phot'][code][aper]['aperture_sum_edge'] != 0).astype(int),}
    )

#derived_photometry_table_(data,12.8,'nn')

In [18]:
def accumulate_photometry(data,code,
                          apers=[1.5, 2.3, 3.8, 6.0, 9.0, 12.8, 17.3]):
    band = {'n':'NUV','f':'FUV'}[code[0]]
    phot = pd.DataFrame()
    for i,aper in enumerate(apers):
        suffix = f'_APER{i}'
        phot[f'{band}_SUM{suffix}']=list(data['phot'][code][aper]['aperture_sum'])
        phot[f'{band}_EDGE{suffix}']=list((data['phot'][code][aper]['aperture_sum_edge'] != 0).astype(int))
        phot[f'{band}_MASK{suffix}']=list((data['phot'][code][aper]['aperture_sum_mask'] != 0).astype(int))
        dp = derived_photometry_table(data,aper,code)
        phot = pd.concat([phot,dp.add_prefix(f'{band}_').add_suffix(suffix)],axis=1)
    phot[f'{band}_EXPT']=np.full(len(data['phot'][code][aper]),
                                 data['expt'][band]['expt'].sum())
    return pd.DataFrame(phot)

# accumulate_photometry(data,'nn')

In [25]:
def make_catalog_by_code(at,c,expts,eclipse):
    band = {'n':'NUV','f':'FUV'}[c[0]]
    cat = pd.concat([pd.DataFrame({'eclipse':np.full(len(at[f'{c}'][12.8]),eclipse),}),
                     at[f'{c}'][12.8][['ra','dec']],
                     at[f'{c}'][12.8][['xcenter','ycenter']].add_prefix(f'{band}_'),
                     accumulate_photometry(at,c,expts),],axis=1)
    if len(set(c))==1: # only the primary / extracted catalog contains extended source information
        cat = pd.concat([cat,pd.DataFrame({f'{band}_EXTENDED':
                                           list(at[f'{c}'][12.8]['extended_source'])})],axis=1)
    return cat

def make_catalog_by_code(data,code):
    band = {'n':'NUV','f':'FUV'}[code[0]]
    cat = pd.concat([pd.DataFrame(
        {'eclipse':np.full(len(data['phot'][code][12.8]),
                           data['eclipse']),}),
                    ],axis=1)
    if len(set(code))==1: # only add ra, dec from primary / extracted catalog
        cat = pd.concat([cat,data['phot'][code][12.8][['ra','dec']],
                        ],axis=1)
    cat = pd.concat([cat,
                     data['phot'][code][12.8][['xcenter','ycenter']].add_prefix(f'{band}_'),
                     accumulate_photometry(data,code),
                    ],axis=1)
    if len(set(code))==1: # only the primary / extracted catalog contains extended source information
        cat = pd.concat([cat,pd.DataFrame({f'{band}_EXTENDED':
                                           list(data['phot'][code][12.8]['extended_source'])})],axis=1)
    return cat

def make_catalog(data,band):
    b = band[0].lower()
    code = f'{b}{b}'
    cat_1 = make_catalog_by_code(data,code)
    c = ({"f":"n","n":"f"}[b])+b # there must be a more elegant way to do this
    cat_2 = make_catalog_by_code(data,code)
    return pd.concat([cat_1,cat_2],axis=1).rename(columns=str.upper)

make_catalog(data,'NUV')

,ECLIPSE,RA,DEC,NUV_XCENTER,NUV_YCENTER,NUV_SUM_APER0,NUV_EDGE_APER0,NUV_MASK_APER0,NUV_CPS_APER0,NUV_CPS_ERR_APER0,...,NUV_EDGE_APER6,NUV_MASK_APER6,NUV_CPS_APER6,NUV_CPS_ERR_APER6,NUV_MAG_APER6,NUV_MAG_ERR_APER6,NUV_MASK_FLAG_APER6,NUV_EDGE_FLAG_APER6,NUV_EXPT,NUV_EXTENDED
0,23456,323.448811,-2.638534,1638.452026,170.996174,74.195186,1,0,0.049456,0.005742,...,1,0,2.074579,0.037187,19.287675,0.019638,0,1,1500.228677,0
1,23456,323.458572,-2.636944,1615.050994,174.814059,35.807291,0,0,0.023868,0.003989,...,1,0,1.513029,0.031757,19.630382,0.023031,0,1,1500.228677,0
2,23456,323.448619,-2.635363,1638.913971,178.608991,114.462236,0,0,0.076297,0.007131,...,1,0,1.988438,0.036406,19.333720,0.020063,0,1,1500.228677,0
3,23456,323.509375,-2.632623,1493.247071,185.185578,50.258974,0,0,0.033501,0.004726,...,0,0,1.583271,0.032486,19.581112,0.022509,0,0,1500.228677,0
4,23456,323.459455,-2.632761,1612.932867,184.853847,33.605723,0,0,0.022400,0.003864,...,0,0,1.428154,0.030854,19.693063,0.023713,0,0,1500.228677,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4835,23456,323.546199,-1.505455,1404.865650,2890.475707,63.903954,0,0,0.042596,0.005329,...,0,1,7.910788,0.072616,17.834451,0.010012,1,0,1500.228677,0
4836,23456,323.548553,-1.504981,1399.217901,2891.612115,80.647389,0,1,0.053757,0.005986,...,0,1,19.405579,0.113732,16.860184,0.006382,1,0,1500.228677,0
4837,23456,323.543522,-1.505130,1411.288694,2891.256176,47.527211,0,0,0.031680,0.004595,...,0,1,4.648662,0.055665,18.411680,0.013080,1,0,1500.228677,0
4838,23456,323.541941,-1.501864,1415.081510,2899.095427,51.393869,0,1,0.034257,0.004779,...,1,1,5.506844,0.060586,18.227743,0.012011,1,1,1500.228677,0


In [26]:
make_catalog(data,'FUV')

,ECLIPSE,RA,DEC,FUV_XCENTER,FUV_YCENTER,FUV_SUM_APER0,FUV_EDGE_APER0,FUV_MASK_APER0,FUV_CPS_APER0,FUV_CPS_ERR_APER0,...,FUV_EDGE_APER6,FUV_MASK_APER6,FUV_CPS_APER6,FUV_CPS_ERR_APER6,FUV_MAG_APER6,FUV_MAG_ERR_APER6,FUV_MASK_FLAG_APER6,FUV_EDGE_FLAG_APER6,FUV_EXPT,FUV_EXTENDED
0,23456,323.509837,-2.632055,1460.456727,196.750320,20.305039,0,0,0.012099,0.002685,...,0,0,0.204258,0.011032,20.544554,0.060286,0,0,1678.19451,0
1,23456,323.456429,-2.626741,1588.506153,209.505800,12.559613,0,0,0.007484,0.002112,...,0,0,0.144476,0.009278,20.920507,0.072067,0,0,1678.19451,0
2,23456,323.480038,-2.621575,1531.901204,221.905105,7.915088,0,0,0.004716,0.001676,...,0,0,0.214445,0.011304,20.491711,0.058797,0,0,1678.19451,0
3,23456,323.589603,-2.621367,1269.209061,222.391674,10.358416,0,0,0.006172,0.001918,...,0,0,0.154366,0.009591,20.848624,0.069644,0,0,1678.19451,0
4,23456,323.434578,-2.615019,1640.896670,237.639575,94.887268,0,0,0.056541,0.005804,...,0,0,0.649414,0.019672,19.288695,0.033397,0,0,1678.19451,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957,23456,323.746538,-1.551791,892.500563,2789.420749,2.219125,0,0,0.001322,0.000888,...,1,0,0.174280,0.010191,20.716881,0.065418,0,1,1678.19451,0
3958,23456,323.332308,-1.538430,1886.335470,2821.518723,44.113045,0,1,0.026286,0.003958,...,0,1,0.408886,0.015609,19.790993,0.042260,1,0,1678.19451,0
3959,23456,323.330821,-1.536653,1889.905329,2825.784542,25.787951,0,0,0.015366,0.003026,...,0,1,0.408294,0.015598,19.792566,0.042291,1,0,1678.19451,0
3960,23456,323.604938,-1.515287,1232.225045,2877.069324,7.935622,0,0,0.004729,0.001679,...,0,0,0.353016,0.014504,19.950515,0.045550,0,0,1678.19451,0
